In [ ]:
# Read data from google drive 
from google.colab import drive
drive.mount('/content/drive')

import os
# path = '/content/drive/MyDrive/projet_recherche'
path = '/content/drive/MyDrive/ENSIIE/M1/S4/PRR-Projet-Recherche/projet_recherche'
os.chdir(path)
# os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Uniform sampling

In [ ]:
from functions import Call_Price_Heston, C_BS, imp_vol
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Example
Call_Price_Heston(100,100,1,0,1,0.08,1,-.5,0.08,alpha=1,L=1000) 

8.836959972067445

In [ ]:
# choose random values from a given interval
def price_set(name, n = 10, S0 = 100):
    import random
    import numpy as np
    import pandas as pd
    
    def generator(n):
      N = int(2*n)
      S_ = np.zeros(N)
      K_ = np.zeros(N)
      m_ = np.zeros(N) # moneyness
      T_ = np.zeros(N) # time to maturity (in year)
      r_ = np.zeros(N) #risk free rate
      rho_ = np.zeros(N) # correlation
      kappa_ = np.zeros(N) # reversion speed
      gam_ = np.zeros(N) # vol of vol
      nu_ = np.zeros(N) # long average variance
      V0_ = np.zeros(N) # initial variance
      price_ = np.zeros(N)
      
      S = S0 # initial stock price
      
      
      for i in range(N):
        m = random.uniform(0.6, 1.4) # moneyness
        K = S/m
        T = random.uniform(0.05, 3) # time to maturity (in year)
        r = random.uniform(0, 0.05) #risk free rate
        kappa = random.uniform(0, 3) # reversion speed
        nu = random.uniform(0.01, 0.5) # long average variance
        gam = random.uniform(0.01, 0.8) # vol of vol
        rho = random.uniform(-0.9, 0) # correlation
        V0 = random.uniform(0.05, 0.5) # initial variance / vol

        price = Call_Price_Heston(S,K,T,r,kappa,nu,gam,rho,V0)
        
        S_[i] = S
        K_[i] = K
        m_[i] = m # moneyness
        T_[i] = T # time to maturity (in year)
        r_[i] = r #risk free rate
        rho_[i] = rho # correlation
        kappa_[i] = kappa  # reversion speed
        gam_[i] = gam # vol of vol
        nu_[i] = nu # long average variance
        V0_[i] = V0 # initial vol
        price_[i] = price

      df = [S_, K_, m_, T_, r_, rho_, kappa_, gam_, nu_, V0_, price_]
      df = pd.DataFrame(df, index=['S', 'K', 'm', 'T', 'r', 'rho', 'kappa', 'gam', 'nu', 'V0', 'price'])
      df = df.T
      df.dropna(axis=0, how='any', inplace = True)
      return df
    
    df = generator(n)
    while len(df)<n : 
      new = generator(n - len(df))
      df = df.append(new, ignore_index=True)

    df = df.head(n)
    df.to_csv(name, index = False)

    
    return df
       

Example of data generation.

In [ ]:
# p = price_set(name ='data/param2_1.csv', n = 10)
# p.head()
# Time : 28s

,S,K,m,T,r,rho,kappa,gam,nu,V0,price
0,100.0,151.943538,0.658139,1.548234,0.000955,-0.058557,1.977661,0.492809,0.025501,0.150168,1.720632
1,100.0,83.827984,1.192919,0.114196,0.045852,-0.632799,2.313996,0.231599,0.149548,0.276901,17.875265
2,100.0,124.299035,0.804511,0.770601,0.042610,-0.250266,0.082220,0.437886,0.491528,0.204895,8.477790
3,100.0,74.782300,1.337215,0.388238,0.035670,-0.437473,2.740561,0.759515,0.401809,0.129190,28.662536
4,100.0,96.666383,1.034486,0.439848,0.042632,-0.692286,0.730711,0.193000,0.466016,0.063102,11.777303


Generating the batches of data and saving them to csv files.

In [ ]:
for i in range(10):
  n = 10000
  price_set(name ='data/param2_'+str(i+1)+'.csv', n = n)
  print('data/param2_'+str(i+1)+'.csv', 'successfuly generated with', n, 'rows.')

data/param2_1.csv successfully generated with 10000 rows.
data/param2_2.csv successfully generated with 10000 rows.
data/param2_3.csv successfully generated with 10000 rows.
data/param2_4.csv successfully generated with 10000 rows.
data/param2_5.csv successfully generated with 10000 rows.
data/param2_6.csv successfully generated with 10000 rows.
data/param2_7.csv successfully generated with 10000 rows.
data/param2_8.csv successfully generated with 10000 rows.
data/param2_9.csv successfully generated with 10000 rows.
data/param2_10.csv successfully generated with 10000 rows.


 # LHS 

In [ ]:
# Functions
import numpy as np
import random
import pandas as pd

'''
The purpose of this document is to:
1. Receive a set of variable range numpy matrices and the number of sample requirements, shape = (m, 2), and output the sample numpy matrix
Execute the ParameterArray function
'''

def Partition (number_of_sample,
               limit_array):
    """
    Divide the variable interval of each variable according to the number of samples, and return the divided variable interval matrix
    :param number_of_sample: the number of samples to output
    :param limit_array: A matrix composed of all variable ranges, which is a (m, 2) matrix, where m is the number of variables, and 2 represents the upper and lower limits
    :return: Return the divided variable interval matrix (three-dimensional matrix), each layer of the three-dimensional matrix corresponds to one variable
    """
    coefficient_lower = np.zeros((number_of_sample, 2))
    coefficient_upper = np.zeros((number_of_sample, 2))
    for i in range(number_of_sample):
        coefficient_lower[i, 0] = 1 - i / number_of_sample
        coefficient_lower[i, 1] = i / number_of_sample
    for i in range(number_of_sample):
        coefficient_upper[i, 0] = 1-(i+1) / number_of_sample
        coefficient_upper[i, 1] = (i+1) / number_of_sample

    partition_lower = coefficient_lower @ limit_array.T  # lower bound
    partition_upper = coefficient_upper @ limit_array.T  # upper bound

    partition_range = np.dstack((partition_lower.T, partition_upper.T))  
    return partition_range 

def Representative(partition_range):
    """
    function to compute a single random representative number
    :param partition_range: A three-dimensional matrix with shape (m,N,2), where m is the number of variables, n is the number of samples, and 2 represents the two columns of the upper and lower limits of the interval
    :return: Returns a matrix composed of random representative numbers of each variable's partitioned interval, with each column representing a variable
    """
    number_of_value = partition_range.shape[0]  # obtain numbre of variables 
    numbers_of_row = partition_range.shape[1]  # number of layers
    coefficient_random = np.zeros((number_of_value,numbers_of_row, 2))  # create random coef matrix
    representative_random = np.zeros((numbers_of_row, number_of_value))

    for m in range(number_of_value):
        for i in range(numbers_of_row):
            y = random.random()
            coefficient_random[m,i, 0] = 1 - y
            coefficient_random[m,i, 1] = y

    temp_arr = partition_range * coefficient_random  # 利用*乘实现公式计算（对应位置进行乘积计算），计算结果保存于临时矩阵 temp_arr 中
    for j in range(number_of_value): #计算每个变量各区间内的随机代表数，行数为样本个数n，列数为变量个数m
        temp_random = temp_arr[j, :, 0] + temp_arr[j, :, 1]
        representative_random[:,j] = temp_random
    return representative_random  # 返回代表数向量

def Rearrange(arr_random):
    """
    Shuffle the data in the columns of the matrix
    :param arr_random: a matrix with N rows and m columns
    :return: shuffled matrix for each column
    """
    for i in range(arr_random.shape[1]):
        np.random.shuffle(arr_random[:, i])
    return arr_random



def ParameterArray(limitArray,
                   sampleNumber):
    """
    According to the input range matrix of each variable and the desired number of samples, output the sample parameter matrix
    :param limitArray: variable upper and lower limit matrix, shape is (m, 2), m is the number of variables
    :param sampleNumber: The number of samples you want to output
    :return: sample parameter matrix
    """
    arr = Partition(sampleNumber, limitArray)
    parametersMatrix = Rearrange(Representative(arr))
    return  parametersMatrix


'''create class'''

class DoE(object):
    def __init__(self, name_value, bounds):
        self.name = name_value
        self.bounds = bounds
        self.type = "DoE"
        self.result = None


class DoE_LHS(DoE):
    # generating samples of LHS
    def __init__(self, name_value, bounds, N):
        DoE.__init__(self, name_value, bounds)
        self.type = "LHS"
        self.ParameterArray = ParameterArray(bounds, N)
        self.N = N

    def write_to_csv(self, name):
        """
        write into LHS.csv and storage
        """
        sample_data = pd.DataFrame(self.ParameterArray, columns=self.name)
        sample_data.to_csv(name, index=False)

'''applications'''

# arr_limit = np.array([[-100, -100, -100, -1000, -1000, -1000, 0, 32, 8, 100],
#                       [100, 100, 100, 1000, 1000, 100, 10, 2000, 100, 500]]).T
# name_value = ["Fx", "Fy", "Fz", "Mx", "My", "Mz", "Pressure", "R", "nozzle_th", "nozzle_h"]  # vars. name
# q = DoE_LHS(N=100, bounds=arr_limit, name_value=name_value)
# # q.write_to_csv() 



'applications'

In [ ]:
# choose n random values from a given interval

def price_set(name, n = 10, S0 = 100):
    # n : number of samples
    # S0 : initial stock price
    # name : name of the file to save
    import pandas as pd

    def generator():
      # N = n
      arr_limit = np.array([[0.6, 0.05, 0, -0.9, 0.00001, 0.010001, 0.010001, 0.050001],
                            [1.4, 3, 0.05, 0, 3, 0.8, 0.5, 0.5]]).T
      name_value = ['m', 'T', 'r', 'rho', 'kappa', 'gamma', 'nu', 'nu0']  # variable names
      q = DoE_LHS(N= 200, bounds=arr_limit, name_value=name_value)
      q.write_to_csv(name) 
      df = pd.read_csv(name)
      
      df['S'] = S0    
      df['K'] = S0/df['m']
      df['price'] = df.apply(lambda row : Call_Price_Heston(row['S'], row['K'], row['T'], row['r'], row['kappa'],row['nu'],row['gamma'], row['rho'],row['nu0']), axis = 1) 
      
      # prix = np.zeros(len(df))
      # for i in range(len(df)): # equivalent to "lambda row"
      #     prix[i] = Call_Price_Heston(df['S'][i], df['K'][i], df['T'][i], df['r'][i], df['kappa'][i], 
      #                                 df['nu'][i],df['gamma'][i], df['rho'][i],df['nu0'][i])
      # df['price'] = prix
      

      # df = df.drop(['K', 'S'], axis=1)
      df.dropna(axis=0, how='any', inplace = True)
      return df
    
    df = generator(n)
    while len(df)<n : 
      new = generator()
      df = df.append(new, ignore_index=True)


    df = df.head(n)
    df.to_csv(name, index = False)
    
    return df


In [ ]:
price_set(name='data/param2.csv', n = 500).head()
# Time : 1min.25s

,m,T,r,rho,kappa,gamma,nu,nu0,S,K,price
0,1.357785,1.002182,0.031340,-0.203035,1.054927,0.492336,0.039166,0.376663,100,73.649350,34.811571
1,0.757901,2.960906,0.040830,-0.597902,0.264853,0.593395,0.019629,0.200691,100,131.943391,11.790365
2,0.971358,2.803367,0.012011,-0.826460,0.093391,0.097565,0.046869,0.310041,100,102.948696,33.351052
3,0.889469,2.892645,0.039078,-0.862914,0.523966,0.390528,0.247491,0.240221,100,112.426689,29.209416
4,0.752286,0.689531,0.049305,-0.222437,0.615804,0.725575,0.317439,0.252951,100,132.928146,7.150266
